<a href="https://colab.research.google.com/github/cjweems/image-filter-project/blob/feedback/Filter_Project_2nd_Version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
import os
import cv2
import matplotlib.pyplot as plt
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode
import PIL
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import numpy as np

# Function to take the photo
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

def faceMask():
  # Open the image
  img = cv2.imread('photo.jpg')

  # Gray image
  gray_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

  #Load the pre-trained classifier
  face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml") #Note the change

  # Preform the face detection
  face = face_cascade.detectMultiScale(gray_img,1.1,4)

  # Make the image color
  img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  # Mask
  mask = np.zeros(img.shape[:2], dtype="uint8")

  for x,y,width,height in face:
    # Find the center of the face
    corner1 = (x,y)
    corner2 = (x+width, y)
    corner3 = (x, y+height)
    corner4 = (x+width, y+height)
    centerx = (x + (x+width) + x + (x+width)) / 4
    centery = (y + (y+height) + y + (y+height)) / 4

    # Draw the ellipse to fit the face
    center_coordinates = (int(centerx), int(centery))
    axesLength = (125, 175)
    cv2.ellipse(mask, center_coordinates, axesLength, 0, 0, 360, (255, 255, 255), -1)

  # Cut out the circle from the image
  faceMask = cv2.bitwise_and(img_rgb, img_rgb, mask=mask)

  plt.imshow(faceMask)
  plt.savefig("masked.jpg")

  backgroundblur = cv2.blur(img_rgb, (25,25))

  plt.imshow(backgroundblur)
  plt.savefig("background.jpg")

def faceOverlay():
  return
# Take the photo
try:
  filename = take_photo()
except Exception as err:
  print(str(err))

faceMask()
faceOverlay()